In [1]:
import psycopg2
import pandas as pd

In [2]:
user = ''
pw=''
host = ''
database = ''
table_name = ''

In [3]:
conn = psycopg2.connect(host=host, database=database, user=user, password=pw)
cur = conn.cursor()

In [8]:
# Creat table in DataBase
conn.commit()
query = """
    DROP TABLE IF EXISTS {table_name};
    CREATE TABLE {table_name}
    (   product character varying(20)
      , nuser BIGINT
      , date timestamp
      , Count BIGINT
      , type  character varying(20)
    )
    DISTRIBUTED BY (product);
    """.format(table_name=table_name)
cur.execute(query)
conn.commit()

In [9]:
df = pd.DataFrame({'product' : ['prodA', 'prodA', 'prodB','prodB'],
                   'nuser': [29, 31, 17, 28],
                   'date' : ['02-27-2013','03-02-2013','02-26-2013','03-02-2013'],
                   'count': [113,152,40,73],
                   'type': ['Document', 'Extraction','Document','Extraction']
                  })
df['date'] = pd.to_datetime(df.date)

In [10]:
df

,product,nuser,date,count,type
0,prodA,29,2013-02-27,113,Document
1,prodA,31,2013-03-02,152,Extraction
2,prodB,17,2013-02-26,40,Document
3,prodB,28,2013-03-02,73,Extraction


In [12]:
# Write dataframe to memory as csv
import io
csv_io = io.StringIO()
df.to_csv(csv_io, sep='\t', header=True, index=False)
csv_io.seek(0)

# Copy the dataframe in memory to GP
conn.commit()
copy_sql = """
           COPY {table_name} FROM stdin WITH CSV HEADER
           DELIMITER as '\t'
           """.format(table_name=table_name)
cur.copy_expert(sql=copy_sql, file=csv_io)
conn.commit()

In [19]:
sql = """
    WITH Pivoted AS
    (
      SELECT *
      FROM {table_name}
      PIVOT 
      ( 
        sum(count) FOR Type IN (Document, Extraction)
      ) AS p
    ) 
    SELECT 
      product,
      SUM(nuser) AS TotalUsers,
      MAX(DAte) AS LatestDate,
      MAX(Document) AS Document,
      MAX(Extraction) AS Extraction
    FROM Pivoted
    GROUP BY Product
""".format(table_name=table_name)
temp = pd.read_sql(sql, conn)
temp

DatabaseError: Execution failed on sql '
    WITH Pivoted AS
    (
      SELECT *
      FROM dt216661.kasina_pivot
      PIVOT 
      ( 
        sum(count) FOR Type IN (Document, Extraction)
      ) AS p
    ) 
    SELECT 
      product,
      SUM(nuser) AS TotalUsers,
      MAX(DAte) AS LatestDate,
      MAX(Document) AS Document,
      MAX(Extraction) AS Extraction
    FROM Pivoted
    GROUP BY Product
': syntax error at or near "("
LINE 8:         sum(count) FOR Type IN (Document, Extraction)
                   ^
